![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx.ai and LangChain Agents to perform sequence of actions

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate usage of LangChain Agents with watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use language models, e.g. `meta-llama/llama-3-70b-instruct`, to create Structured Chat LangChain Agent with memory, where an Agent may perform sequence of actions based on the model responses and assures that LLM uses chat memory to provide the most accurate answer on the user question.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [LangChain integration](#watsonxllm)
- [Prompt template and custom tools](#prompt_template)
- [LangChain Agent](#agent)
- [Example of Structured Chat](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependecies

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install langchain | tail -n 1
!pip install langchain-community | tail -n 1
!pip install langchain-ibm | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [1]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your WML api key (hit enter): "),
)

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="watsonxllm"></a>
## Initialize `WatsonxLLM` instance

#### Set model parameters

To set the model parameters we use `GenTextParamsMetaNames` metanames from `ibm-watsonx-ai`.

In [3]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
from langchain_ibm import WatsonxLLM

Based on the performed experiments we found that the following parameters work well with the LangChain Agents created in the further section.

In [4]:
parameters = {
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 200,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 10,
    GenTextParamsMetaNames.TEMPERATURE: 0.1,
    GenTextParamsMetaNames.TOP_K: 20,

}

Furthermore, you need to specify `model_id` that will be used for inferencing. In the following notebook we use `meta-llama/llama-3-70b-instruct`.

In [5]:
llm_chat = WatsonxLLM(url=credentials.get('url'),
                      apikey=credentials.get('apikey'),
                      project_id=project_id,
                      model_id="meta-llama/llama-3-70b-instruct",
                      params=parameters
                    )

In [6]:
response = llm_chat.invoke('What is a generative AI? Answer briefly.')
print(response)

 Generative AI is a type of artificial intelligence that can create new, original content, such as images, videos, music, or text, based on patterns and structures it has learned from existing data.


<a id="prompt_template"></a>
## Prompt template preparation and tools
### Create Prompt Template

A structured chat prompt template is composed of several sections: system prompt, chat history and user input. The following prompt template was prepared based on the LangChain [documentation](https://python.langchain.com/docs/modules/agents/agent_types/structured_chat) and adjusted to work with the chat models available on watsonx.ai.

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

In [8]:
system_prompt = """Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. 

AI assistant has access to the following tools:

{tools}

Use a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). 

Use only valid "action" values: "Final Answer" or {tool_names} and provide only one action per $JSON_BLOB. Use "action" value "Final Answer" if you know what to respond. Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools only if necessary. Format is Action:```$JSON_BLOB```then Observation
"""

In [9]:
user_input = '{input}\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what and use tools only if necessary)'

To create the final structured chat prompt template one can use initialization class method from `ChatPromptTemplate` .

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", user_input),
    ]
)

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. \n\nAI assistant has access to the following tools:\n\n{tools}\n\nUse a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). \n\nUse only valid "action" values: "Final Answer" or {tool_names} and provide only one action per $JSON_BLOB. Use "action" value "Final Answer" if you know what to respond. Provide only ONE action per $JSON_BLOB, as shown:\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\nFollow this format:\nQuestion: input question to answer\nThought: consider previous and subsequent steps

### Prepare tools

In this section we prepare tools (python functions) that the AI assistant will have access to.

In [12]:
from langchain.agents import tool
from langchain.tools import StructuredTool

In [13]:
@tool
def get_word_length(word: str) -> int:
    """Return the length of the given word."""
    return len(word)

@tool
def remove_punctuations(text: str) -> str:
    """Remove all punctuation symbols from the text."""
    import re 

    new_string = re.sub(r'[^\w\s]','',text) 
    return new_string

@tool
def get_mean_from_sequence(sequence: str) -> float:
    """Return the mean value from the sequence of numbers."""
    import numpy
    return numpy.mean(numpy.array(sequence.split(','), dtype=float))

## Structured tools

def get_normalize_vector(vector: list[float]) -> str:
    """Return a normalized `vector`."""
    import numpy
    return str(list(vector / numpy.sqrt(numpy.sum(numpy.array(vector)**2))))

get_normalize_vector_structured = StructuredTool.from_function(
    func=get_normalize_vector,
    name="Normalize vector",
    description="useful when want to normalize a vector",
)

def get_gcd(a: int, b: int) -> int:
    """Return the greatest common divisor of the two integers."""
    import math
    return math.gcd(a, b)

get_gcd_structured = StructuredTool.from_function(
    func=get_gcd,
    name="Greatest common divisor",
    description="useful when want to find greatest common divisor of the two integers",
)

tools_chat = [get_word_length, get_mean_from_sequence, remove_punctuations, get_normalize_vector_structured, get_gcd_structured]

<a id="agent"></a>
## Initialize an Agent

Following section is the central part of the presented notebook. Here, we construct LangChain Agent and initialise `AgentExecutor` which manage conversation with AI assistant, that has access to the predefined tools, and human.

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser

In the prompt template, we include the tools signature and their description.

In [15]:
prompt_chat = prompt.partial(
        tools=[str(convert_to_openai_tool(func)) 
               if isinstance(func, StructuredTool) 
               else render_text_description([func]) 
               for func in tools_chat],
        tool_names=", ".join([t.name for t in tools_chat]),
    )

Finally, when we have all components prepared we can define Chat Agent and initialize `AgentExecutor`.

In [16]:
# Setting stop sequence for LLM
llm_with_stop = llm_chat.bind(stop=["Observation:"])

agent_chat = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt_chat
    | llm_with_stop
    | JSONAgentOutputParser()
)

agent_executor_chat = AgentExecutor(
    agent=agent_chat, tools=tools_chat, verbose=True, handle_parsing_errors=True
)

Notice that we use `handle_parsing_errors=True` to ensure that when LLM is not able to determine what step to take because its output does not have the correct structure to be handled  by output parser, the user's input will be sent back to the model with the appropriate information to improve the model reponse. Setitng this parameter to True help to keep chat conversation smooth, but may also result in multiple requests with many tokens being sent to the model and back.

<a id="chat"></a>
## Structured Chat with memory

To include chat memory in conversation with AI assistant we are going to use `ChatMessageHistory` that simply store chat history in the prompt and send it with every user question to the LLM. 

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [18]:
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor_chat,
    get_session_history=lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

The `session_id` is needed to persist conversation histories. Note that watsonx.ai does not conversation history. Let us start the chat!

In [21]:
response = agent_with_chat_history.invoke(
    {"input": "Hi! How are you?"},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']



> Entering new AgentExecutor chain...
 

Question: Hi! How are you?
Thought: I've seen this question before. I should respond politely.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you for asking!"
}
```
Observation:

> Finished chain.


"I'm doing well, thank you for asking!"

In [22]:
response = agent_with_chat_history.invoke(
    {"input": "Normalize the following vector: `1, 2, 4`."},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']



> Entering new AgentExecutor chain...
 

Question: Normalize the following vector: `1, 2, 4`.
Thought: I need to normalize the vector.
Action:
```
{
  "action": "Normalize vector",
  "action_input": {"vector": [1, 2, 4]}
}
```
Observation:[0.2182178902359924, 0.4364357804719848, 0.8728715609439696] 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The normalized vector is [0.2182178902359924, 0.4364357804719848, 0.8728715609439696]."
}
```

> Finished chain.


'The normalized vector is [0.2182178902359924, 0.4364357804719848, 0.8728715609439696].'

In [23]:
response = agent_with_chat_history.invoke(
    {"input": "What is the greatest common divisor of integers 1215 and 645?"},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']



> Entering new AgentExecutor chain...
 
Question: What is the greatest common divisor of integers 1215 and 645?
Thought: I need to use the Greatest common divisor tool
Action:
```
{
  "action": "Greatest common divisor",
  "action_input": {"a": 1215, "b": 645}
}
```
Observation:15 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The greatest common divisor of integers 1215 and 645 is 15."
}
```

> Finished chain.


'The greatest common divisor of integers 1215 and 645 is 15.'

In [24]:
response = agent_with_chat_history.invoke(
    {"input": "What is a median?"},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']

Parent run 2dded844-5d50-4621-9b28-6edbb8e5dc09 not found for run 049d302f-39de-4728-a3e5-03519d49f06f. Treating as a root run.




> Entering new AgentExecutor chain...
 

Question: What is a median?
Thought: I don't know the answer, but I can try to explain it in simple terms.
Action:
```
{
  "action": "Final Answer",
  "action_input": "A median is a middle value in a sequence of numbers, where half the numbers are above it and half are below it. However, I'm not sure how to calculate it. Can you provide more context or clarify what you mean?"
}
```
Observation:

> Finished chain.


"A median is a middle value in a sequence of numbers, where half the numbers are above it and half are below it. However, I'm not sure how to calculate it. Can you provide more context or clarify what you mean?"

In [25]:
response = agent_with_chat_history.invoke(
    {"input": "Could you be more specific and provide some example?"},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']

Parent run 72101dec-1497-4b92-9034-033a19d0fee8 not found for run b7852e6d-b956-4a32-afc4-b1e2254b7247. Treating as a root run.




> Entering new AgentExecutor chain...
 

Question: Could you be more specific and provide some example?
Thought: I need to provide a clear explanation with an example
Action:
```
{
  "action": "get_mean_from_sequence",
  "action_input": {"sequence": "1, 2, 3, 4, 5"}
}
```
Observation:3.0 
Action:
```
{
  "action": "Final Answer",
  "action_input": "To calculate the median, we need to arrange the numbers in order. If we have an odd number of values, the middle value is the median. If we have an even number of values, the median is the average of the two middle values. For example, in the sequence 1, 2, 3, 4, 5, the median is 3, since it's the middle value. In the sequence 1, 2, 3, 4, the median is (2 + 3) / 2 = 2.5, since it's the average of the two middle values."
}
```

> Finished chain.


"To calculate the median, we need to arrange the numbers in order. If we have an odd number of values, the middle value is the median. If we have an even number of values, the median is the average of the two middle values. For example, in the sequence 1, 2, 3, 4, 5, the median is 3, since it's the middle value. In the sequence 1, 2, 3, 4, the median is (2 + 3) / 2 = 2.5, since it's the average of the two middle values."

Let us now use other tools that were provided.

In [27]:
response = agent_with_chat_history.invoke(
    {"input": "Use one of the tool and remove all punctuation symbols from the sentence: `What, is a. loan?!`. Return clear sentence."},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']

Parent run f5361457-fe83-4dc1-b1c7-1a3f7526f149 not found for run 949b4fc4-c133-455a-909c-712e0388bde3. Treating as a root run.




> Entering new AgentExecutor chain...
 
Question: Use one of the tool and remove all punctuation symbols from the sentence: `What, is a. loan?!`. Return clear sentence.
Thought: I can use the remove_punctuations tool to remove all punctuation symbols from the sentence.
Action:
```
{
  "action": "remove_punctuations",
  "action_input": {"text": "What, is a. loan?!"}
}
```
Observation:What is a loan 
Action:
```
{
  "action": "Final Answer",
  "action_input": "What is a loan"
}
```

> Finished chain.


'What is a loan'

In [24]:
response = agent_with_chat_history.invoke(
    {"input": "What is the average length of words `watsonx.ai` and `IBM`?"},
    config={"configurable": {"session_id": "watsonx"}},
)
response['output']



> Entering new AgentExecutor chain...
 
Question: What is the average length of words `watsonx.ai` and `IBM`?
Thought: I need to calculate the length of each word and then find the mean.
Action:
```
{
  "action": "get_word_length",
  "action_input": {"word": "watsonx.ai"}
}
```
Observation:10 
Action:
```
{
  "action": "get_word_length",
  "action_input": {"word": "IBM"}
}
```
Observation:3 
Action:
```
{
  "action": "get_mean_from_sequence",
  "action_input": {"sequence": "10,3"}
}
```
Observation:6.5 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The average length of words `watsonx.ai` and `IBM` is 6.5."
}
```

> Finished chain.


'The average length of words `watsonx.ai` and `IBM` is 6.5.'

History of the chat is stored locally in `message_history` list.

In [25]:
message_history.messages

[HumanMessage(content='Hi! How are you?'),
 AIMessage(content="I'm doing well, thank you for asking!"),
 HumanMessage(content='Hi! How are you?'),
 AIMessage(content="I'm doing well, thank you for asking!"),
 HumanMessage(content='Normalize the following vector: `1, 2, 4`.'),
 AIMessage(content='The normalized vector is [0.2182178902359924, 0.4364357804719848, 0.8728715609439696].'),
 HumanMessage(content='What is the greatest common divisor of integers 1215 and 645?'),
 AIMessage(content='The greatest common divisor of integers 1215 and 645 is 15.'),
 HumanMessage(content='What is the average length of words `watsonx.ai` and `IBM`?'),
 AIMessage(content='The average length of words `watsonx.ai` and `IBM` is 6.5.')]

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use LangChain Agents using custom llm WastonxLLM.
 
Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors:
 **Mateusz Świtała**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.